In [1]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
#读取数据
transform = transforms.Compose(
    [transforms.Resize((32,32)),
     transforms.ToTensor()])

train_data = datasets.MNIST(root="./data", train=True, 
                            download=True,transform=transform)
test_data = datasets.MNIST(root="./data", train=False, 
                           download=True, transform=transform)

In [3]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers = 0)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, num_workers = 0)

In [4]:
# 查看数据
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)
print(example_targets)
print(example_data.shape)

tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8])
torch.Size([32, 1, 32, 32])


In [5]:
import torch.nn as nn
import torch.nn.functional as F
class LeNet(nn.Module):   
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
model = LeNet()
print(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [7]:
import torch.optim as optim
loss_function = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs=10

In [8]:
import time


In [9]:

total_step = len(train_loader)

for epoch in range(epochs):
    
    t1 = time.perf_counter()
    train_loss = 0.0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output,target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
    train_loss = train_loss / len(train_loader.dataset)
    print('Epoch:  {}  \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))
    
    print(time.perf_counter() - t1)

Epoch:  1  	Training Loss: 0.196100
15.2012189
Epoch:  2  	Training Loss: 0.136954
15.267360799999992
Epoch:  3  	Training Loss: 0.125297
15.305150799999993
Epoch:  4  	Training Loss: 0.119851
15.283639199999996
Epoch:  5  	Training Loss: 0.112483
15.3223481
Epoch:  6  	Training Loss: 0.113568
15.356086000000005
Epoch:  7  	Training Loss: 0.110624
15.53653060000002
Epoch:  8  	Training Loss: 0.105304
15.572535099999982
Epoch:  9  	Training Loss: 0.094743
15.667680599999983
Epoch:  10  	Training Loss: 0.100870
15.49976620000001


In [10]:
correct = 0
total = 0
with torch.no_grad():  # 训练集中不需要反向传播
    for data in test_loader:
        images, labels = data
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

Accuracy of the network on the test images: 97 %
